<a href="https://colab.research.google.com/github/justinliu23/DME-project/blob/main/OCT_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import shutil
from pathlib import Path
import pandas as pd
import random

import numpy as np
import matplotlib.pyplot as plt
import cv2

In [ ]:
# Authorize access to mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def load_data(dir):
    img_paths = []

    for root, dirs, files in os.walk(dir):
        for filename in files:
            abs_path = os.path.abspath(os.path.join(root, filename))
            img_paths.append(abs_path)

    return img_paths

In [ ]:
source_dir = r'/content/drive/MyDrive/Colab Notebooks/Diabetic Retinopathy/OCT_Dataset_extracted/images_20K'

df1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Diabetic Retinopathy/OCT_Dataset_extracted/Abnormal OCT Scans.csv")
dme_file_list1 = df1.iloc[:, 0].tolist()
# for i in range(len(dme_file_list1)):
#   dme_file_list1[i] = dme_file_list1[i] + r'.jpg'
# append the first file from the csv into the list, because pandas skips it
dme_file_list1.insert(0, '609040_839547_9737192')

df2 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Diabetic Retinopathy/OCT_Dataset_extracted/Abnormal OCT Scans 2.csv")
dme_file_list2 = df2.iloc[:, 0].tolist()
for i in range(len(dme_file_list2)):
  if r'.jpg' in dme_file_list2[i]: # standardize all filenames to not include .jpg
    dme_file_list2[i] = dme_file_list2[i].replace(".jpg", "")
# append the first file from the csv into the list, because pandas skips it
dme_file_list2.insert(0, '100334_282444_1895586_NA')

df3 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Diabetic Retinopathy/OCT_Dataset_extracted/Abnormal OCT Scans - Missing Images.csv")
dme_file_list3 = df3.iloc[:, 0].tolist()
for i in range(len(dme_file_list3)):
  if r'.jpg' in dme_file_list3[i]:
    dme_file_list3[i] = dme_file_list3[i].replace(".jpg", "")
# append the first file from the csv into the list, because pandas skips it
dme_file_list3.insert(0, '609040_839547_9737192')

# prevents duplicates
dme_file_list = list(set(dme_file_list1 + dme_file_list2 + dme_file_list3))

print(len(dme_file_list1))
print(len(dme_file_list2))
print(len(dme_file_list3))
print(len(dme_file_list))
print(dme_file_list)

In [ ]:
source_dir = r'/content/drive/MyDrive/Colab Notebooks/Diabetic Retinopathy/OCT_Dataset_extracted/images_20K'

all_img_paths = load_data(source_dir)
print('# img paths:', len(all_img_paths))
print(all_img_paths)

In [ ]:
source_dir = r'/content/drive/MyDrive/Colab Notebooks/Diabetic Retinopathy/OCT_Dataset_extracted/Abnormal'

ab_img_paths = load_data(source_dir)
print('# img paths:', len(ab_img_paths))
print(ab_img_paths)

In [ ]:
shortened_all_img_paths = []

for i, img_path in enumerate(all_img_paths):
    path_after_dir = r'/'.join(Path(img_path).parts[8:])
    # path_after_dir = os.path.splitext(path_after_dir)[0]
    shortened_all_img_paths.append(path_after_dir.replace('.jpg', ''))

print(len(shortened_all_img_paths))
print(shortened_all_img_paths)

In [ ]:
shortened_ab_img_paths = []

for i, img_path in enumerate(ab_img_paths):
    path_after_dir = r'/'.join(Path(img_path).parts[8:])
    # path_after_dir = os.path.splitext(path_after_dir)[0]
    path_after_dir = path_after_dir.replace(' - Copy.jpg', '')
    path_after_dir = path_after_dir.replace('.jpg', '')
    shortened_ab_img_paths.append(path_after_dir)

print(len(shortened_ab_img_paths))
print(shortened_ab_img_paths)

In [ ]:
dme_file_list = list(set(dme_file_list + shortened_ab_img_paths))
print(len(dme_file_list))
print(dme_file_list)

In [ ]:
# normals = []
# for i in shortened_all_img_paths:
#   found = False
#   for j in dme_file_list:
#     if j in i: # some labels on spreadsheet weren't full filepath of file, so check as a substring (~string.contains)
#       found = True
#   if not found:
#     normals.append(i)

# print(normals)
# print(len(normals))

In [ ]:
normals = []
for i in shortened_all_img_paths:
  found = False
  for j in dme_file_list:
    if j in i: # some labels on spreadsheet weren't full filepath of file, so check as a substring (~string.contains)
      found = True
  if not found:
    normals.append(i)

print(normals)
print(len(normals))

In [ ]:
source_dir = r'/content/drive/MyDrive/Colab Notebooks/Diabetic Retinopathy/OCT_Dataset_extracted/images_20K'
dest_dir = r'/content/drive/MyDrive/Colab Notebooks/Diabetic Retinopathy/OCT_Dataset_extracted/Train/dme'

count = 0
for fp in dme_file_list:
  if os.path.exists(os.path.join(source_dir, fp + r'.jpg')):
    count += 1
    shutil.copy(os.path.join(source_dir, fp + r'.jpg'), dest_dir)
  # else:
  #   print(fp) # prints missing image filenames

print(count) # num of found image files

In [ ]:
dest_dir = r'/content/drive/MyDrive/Colab Notebooks/Diabetic Retinopathy/OCT_Dataset_extracted/Train/normal'

# we have ~650 DME OCT images, so we only need ~650 normal controls to create a balanced dataset
for fp in random.sample(normals, 800): # random selection w/out duplicates
  if os.path.exists(os.path.join(source_dir, fp + r'.jpg')):
    # print(os.path.join(source_dir, fp))
    shutil.copy(os.path.join(source_dir, fp + r'.jpg'), dest_dir)



---



In [ ]:
def extract_oct_slices(file_paths, new_dir, plot=False):
  BBOX_CONSTANT = 7 # pixel constant that removes the thin, outer white frame from bounding box segmentations
  # SEGMENTATION_THRESHOLD = 220
  DARKNESS_THRESHOLD = 50
  MAX_BOXES = 15 # max num of rectanlges that will be detected when finding the boxes with largest area (the slices)

  ROIs = []
  ROI_images = []

  for img_path in file_paths:
    img = cv2.imread(img_path)
    img_copy = img.copy()
    gray_img = cv2.cvtColor(img_copy, cv2.COLOR_BGR2GRAY)
    # threshold_value, binary_img = cv2.threshold(gray_img, SEGMENTATION_THRESHOLD, 255, 0)
    threshold_value, binary_img = cv2.threshold(gray_img, 190, 255, cv2.THRESH_BINARY) # 150 instead of 190 also works well
    # plt.imshow(binary_img)
    # plt.show()

    contours, hierarchy = cv2.findContours(binary_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    areas = [cv2.contourArea(contour) for contour in contours]

    slices = 0
    
    for i in range(MAX_BOXES):
      max_index = np.argmax(areas)
      contour = contours[max_index]

      x, y, w, h = cv2.boundingRect(contour)
      xmin = x + BBOX_CONSTANT
      xmax = x + w - BBOX_CONSTANT
      ymin = y + BBOX_CONSTANT
      ymax = y + h - BBOX_CONSTANT

      ROI = img_copy[ymin:ymax, xmin:xmax]

      pxl_mean = np.mean(ROI)
      # print('pxl_mean:', pxl_mean)
      if pxl_mean < DARKNESS_THRESHOLD: # only the OCT images have an overall pixel mean < 50 (mostly black)
        slices += 1
        ROIs.append(ROI)

        # uncomment to draw green bboxes around the extracted slices --> saves to ROI_images
        # cv2.rectangle(img_copy, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

        if plot:
          plt.imshow(ROI)
          plt.show()

        path_after_dir = r'/'.join(Path(img_path).parts[8:])
        # don't use the below line because some filenames have '.' before the filetype (ex: id_tag_#.Others.png)
        # path_after_dir = path_after_dir[:str(path_after_dir).index('.')] + r'_slice_{}.png'.format(slices)
        path_after_dir = str(path_after_dir)[:-4] + r'_slice_{}.png'.format(slices)
        final_img_path = Path(os.path.join(new_dir, path_after_dir))
        final_img_dir = final_img_path.parent
        
        # print('path_after_dir', path_after_dir)
        # print('final_img_path', final_img_path)
        # print('final_img_dir', final_img_dir)
        # print('---------')

        if os.path.isdir(str(final_img_dir)) == False:
            os.makedirs(final_img_dir)
            print('New directory made:', final_img_dir)

        cv2.imwrite(str(final_img_path), ROI)

      # remove the current contour (w/ greatest area) to prevent it from being detected on the next iteration
      areas.remove(areas[max_index])
      contours.pop(max_index)

    ROI_images.append(img_copy)

    if plot: # show whole image on last iteration
      plt.imshow(img_copy)
      plt.show()

  return ROIs, ROI_images

In [ ]:
dir = r'/content/drive/MyDrive/Colab Notebooks/Diabetic Retinopathy/OCT_Dataset_extracted/OCT_Dataset'
new_dir = r'/content/drive/MyDrive/Colab Notebooks/Diabetic Retinopathy/OCT_Dataset_extracted/OCT_Dataset_extractions'

img_paths = load_data(dir)
print('# img_paths:', len(img_paths))
print(img_paths)

In [ ]:
ROIs, ROI_images = extract_oct_slices(img_paths, new_dir, plot=False)
print('Total # Slices Extracted:', len(ROIs))

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


New directory made: /content/drive/MyDrive/Colab Notebooks/Diabetic Retinopathy/OCT_Dataset_extracted/OCT_Dataset_extractions/Train/normal
Total # Slices Extracted: 9539
